In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd ../..

/home/hazzu/Code/opencv_streamlit


In [5]:
import numpy as np


def get_precision_and_recall(ground_truth: np.ndarray, pred: np.ndarray, threshold=5):
    """
    Calculate precision and recall of the predicted points.

    Input:
    - ground_truth: np.ndarray of shape (N, 2) containing the ground truth points.
    - pred: np.ndarray of shape (M, 2) containing the predicted points.

    Output:
    - (precision, recall): Tuple of floats containing the precision and recall.
    """

    if len(ground_truth) == 0 or len(pred) == 0:
        return (0, 0)

    tp, fn, fp = 0, 0, 0
    for point in pred:
        distance = np.linalg.norm(ground_truth - point, axis=1)
        if np.any(distance <= threshold):
            tp += 1
        else:
            fp += 1

    for point in ground_truth:
        distance = np.linalg.norm(pred - point, axis=1)
        if not np.any(distance <= threshold):
            fn += 1

    if tp + fp == 0 or tp + fn == 0:
        return (0, 0)
    return (tp / (tp + fp), tp / (tp + fn))

In [9]:
import os, cv2
import numpy as np
from tqdm import tqdm
from services.semantic_keypoint_detection.services import DATATYPES

sift = cv2.SIFT_create()
orb = cv2.ORB_create()

pr_sift, pr_orb = [], []
for i in range(8):
    _pr_sift, _pr_orb = [], []
    for j in tqdm(range(500), f"Calculating PR of shape {i}"):
        image = cv2.imread(os.path.join(DATATYPES[i], "images", f"{j}.png"))
        ground_truth = np.load(os.path.join(DATATYPES[i], "points", f"{j}.npy"))
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        pred_sift = sift.detect(gray, None)
        pref_orb = orb.detect(gray, None)

        pred_sift = np.array([[kp.pt[1], kp.pt[0]] for kp in pred_sift])
        pref_orb = np.array([[kp.pt[1], kp.pt[0]] for kp in pref_orb])

        _pr_sift.append(get_precision_and_recall(ground_truth, pred_sift))
        _pr_orb.append(get_precision_and_recall(ground_truth, pref_orb))

    pr_sift.append(_pr_sift)
    pr_orb.append(_pr_orb)

Calculating PR of shape 7: 100%|██████████| 500/500 [00:01<00:00, 338.69it/s]


In [10]:
print(np.shape(pr_sift))
print(np.shape(pr_orb))

(8, 500, 2)
(8, 500, 2)


In [11]:
import os

np.save(os.path.join("./services/semantic_keypoint_detection", "pr_sift.npy"), pr_sift)
np.save(os.path.join("./services/semantic_keypoint_detection", "pr_orb.npy"), pr_orb)